In [1]:
import pandas as pd
import numpy as np

from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.run_query import run_query

In [3]:
sparql_query = """
SELECT (COUNT(DISTINCT ?p) as ?numPredicates)
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
  ?s ?p ?o .
}
"""
run_query(sparql_query, as_dataframe=True)

,numPredicates.value
0,259


In [32]:
sparql_query = """
SELECT (COUNT(DISTINCT ?s) as ?numSubjects)
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
  ?s ?p ?o .
}
"""
run_query(sparql_query, as_dataframe=True)

,numSubjects.value
0,10486792


In [31]:
sparql_query = """
SELECT (COUNT(?o) as ?numObjects)
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
  ?s ?p ?o .
}
"""
run_query(sparql_query, as_dataframe=True)

,numObjects.value
0,244081830


In [29]:
sparql_query = """
SELECT ?p (COUNT(DISTINCT ?s) as ?numSubjects) (COUNT(?o) as ?numObjects)
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
  ?s ?p ?o .
}
GROUP BY ?p
"""
num_sub_obj_group_pred = run_query(sparql_query, as_dataframe=True)

In [30]:
pd.set_option('display.max_rows', 500)
num_sub_obj_group_pred.sort_values(by=['p.value'], ascending=False)

,p.value,numSubjects.value,numObjects.value
37,https://www.w3.org/2004/02/skos/core#narrowMatch,41,55
115,https://www.w3.org/2004/02/skos/core#exactMatch,1304,1580
25,https://schema.org/url,9128523,9128523
108,https://schema.org/reviewCount,2138243,2138243
83,https://schema.org/publicAccess,150346,150346
92,https://schema.org/postalCode,150346,150346
102,https://schema.org/longitude,150346,150346
94,https://schema.org/location,141799,141799
112,https://schema.org/legalName,2138211,2138211
84,https://schema.org/latitude,150346,150346


In [8]:
sparql_query = """
SELECT ?class (COUNT(DISTINCT ?s) as ?numSubjects)
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
  ?s rdfs:Class ?class .
  ?s ?p ?o .
}
GROUP BY ?class
"""
num_sub_group_class = run_query(sparql_query, as_dataframe=True)

In [22]:
pd.set_option('display.max_colwidth', 500)
num_sub_group_class.sort_values(by=['class.value'], ascending=False)

,class.value,numSubjects.value
5,https://schema.org/UserReview,6990280
1,https://schema.org/Person,1987897
3,https://schema.org/ParkingFacility,88814
8,https://schema.org/OpeningHoursSpecification,127123
11,https://schema.org/LocationFeatureSpecification,57962
4,https://schema.org/LocalBusiness,150346
9,https://schema.org/FoodService,28784
2,https://purl.archive.org/purl/yelp/ontology#yelpCategory,1318
7,https://purl.archive.org/purl/yelp/ontology#schemaCategory,229
6,https://purl.archive.org/purl/yelp/ontology#datasetCategory,1311


In [33]:
sparql_query = """
SELECT ?p ?class (COUNT(DISTINCT ?s) as ?numSubjects)
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
  ?s rdfs:Class ?class .
  ?s ?p ?o .
}
GROUP BY ?p ?class
"""
num_sub_group_pred_class = run_query(sparql_query, as_dataframe=True)

In [34]:
num_sub_group_pred_class.sort_values(by=['class.value'], ascending=False)

,p.value,class.value,numSubjects.value
40,https://schema.org/dateCreated,https://schema.org/UserReview,6990280
34,https://schema.org/about,https://schema.org/UserReview,6990280
17,https://schema.org/author,https://schema.org/UserReview,6990280
13,https://schema.org/url,https://schema.org/UserReview,6990280
14,http://www.w3.org/2000/01/rdf-schema#Class,https://schema.org/UserReview,6990280
3,https://schema.org/description,https://schema.org/UserReview,6990280
4,https://schema.org/aggregateRating,https://schema.org/UserReview,6990280
18,https://schema.org/url,https://schema.org/Person,1987897
15,https://schema.org/dateCreated,https://schema.org/Person,1987897
21,http://www.w3.org/2000/01/rdf-schema#Class,https://schema.org/Person,1987897


In [37]:
sparql_query = """
SELECT (COUNT(DISTINCT ?s) as ?numSubjects) (COUNT(DISTINCT ?checkin) as ?numcheckins)
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
  ?s schema:checkinTime ?checkin .
}
"""
checkin = run_query(sparql_query, as_dataframe=True)

In [38]:
checkin

,numSubjects.value,numcheckins.value
0,131930,12887702


In [49]:
sparql_query = """
SELECT DISTINCT ?category
FROM <http://www.yelpkg.com/yelp_kg>
WHERE {
   ?category rdfs:Class <https://purl.archive.org/purl/yelp/ontology#yelpCategory> .

}
"""
run_query(sparql_query, as_dataframe=True)

,category.value
0,https://purl.archive.org/purl/yelp/business_categories#pita
1,https://purl.archive.org/purl/yelp/business_categories#Himalayan
2,https://purl.archive.org/purl/yelp/business_categories#Hypnosis
3,https://purl.archive.org/purl/yelp/business_categories#Hypnotherapy
4,https://purl.archive.org/purl/yelp/business_categories#Nepalese
...,...
1313,https://purl.archive.org/purl/yelp/business_categories#fencing_club
1314,https://purl.archive.org/purl/yelp/business_categories#dialysis_clinic
1315,https://purl.archive.org/purl/yelp/business_categories#hospitalist
1316,https://purl.archive.org/purl/yelp/business_categories#natural_gas_supplier
